# Classes in Python

## What is a class?

Many things :-/

There are several ways to explain classes, some rather abstract and philosophical, others more practical. We don't have much time, so I go for *very* practical with a few examples.

## Classes by example: the class of pets

A class describes data ("attributes") and behavior ("methods") of objects that are "instances" of it:

In [1]:
class Pet:
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        
    def advance_time(self):
        self.age += 1
        self.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def hungrier_than(self, other_pet):
        return self.hunger > other_pet.hunger

my_degu = Pet(name="Vitali")
my_other_degu = Pet("Wladimir")

This is how we can read the above code:

* `my_degu` and `my_other_degu` are objects, and instances of `Pet` (short: "they are pets").
* `Pet` can be understood in several ways: for example as "the concept of pet", or "the entirety of pets", or "the data type pet".
* `Pet` describes some properties and state of a pet (through the attributes `name`, `age`, `hunger`), what happens when a pet is fed (through the method `feed()`), what happens when time passes (through the method `advance_time()`), and how to compare a pet to another pet in some way (through the method `hungrier_than()`).
* `__init__` is a special method: it is called when a new Pet is created. This method is called "the constructor" of the class.
* Every method's first parameter is a reference to the object itself. By convention, it is called "self".

Let's do something with the pets:

In [2]:
for _ in range(10):
    my_degu.advance_time()
    my_other_degu.advance_time()

my_degu.feed()
print(my_other_degu.hungrier_than(my_degu))
print(my_degu.name, my_degu.age)

True
Vitali 10


You see that you can use `my_degu` and `my_other_degu` like you have used any other objects before: by accessing attributes and methods usig "`.`", and by passing objects to functions as a value.

## Class attributes and class methods

A class can not only describe data and behavior of its instances, but also of itself ("the entirety of pets", or "the concept of pets").

We redefine the Pet class a bit:

In [3]:
class Pet:
    population = set()
    
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        self.__class__.population.add(self)
        
    def die(self):
        print("{} dies :(".format(self.name))
        self.__class__.population.remove(self)
        
    def is_alive(self):
        return self in self.__class__.population
        
    @classmethod
    def advance_time(cls):
        for individual in cls.population:
            individual.age += 1
            individual.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def hungrier_than(self, other_pet):
        return self.hunger > other_pet.hunger

my_degu = Pet(name="Vitali")
my_other_degu = Pet("Wladimir")

This is how we can read what is new and exciting about our redefined class:

* `population` is a class attribute, i.e., it is defined on the class. `Pet.population` is the set of all pets. When a pet is created, it is added to the Pet population. When it `die()`s (a method we added), it is removed from the Pet population. For a given pet, we can check whether it's alive by calling `is_alive()`.
* Unlike `population`, the other attributes are instance attributes. `Pet.name` does not exist, but `my_other_degu.name` does.
* In instance methods like `__init__()` or `die()`, `self.__class__` refers to the class of self. We could just as well write `Pet` here, but it's better not to (mor on that later if we have time).
* `advance_time` is now a "class method". Its first parameter is not an instance, but the class itself. The parameter is usually called "cls".

In [4]:
for _ in range(10):
    Pet.advance_time()

my_degu.feed()
print(my_other_degu.hungrier_than(my_degu))
print(my_degu.name, my_degu.age)
print("There are {} pets.".format(len(Pet.population)))

my_degu.die()
print("There are {} pets.".format(len(Pet.population)))
my_other_degu.die()
print("There are {} pets.".format(len(Pet.population)))

True
Vitali 10
There are 2 pets.
Vitali dies :(
There are 1 pets.
Wladimir dies :(
There are 0 pets.


The only big change in the usage example is that instead of having to "advance time" for each pet individually, we now advance time for all pets in existence through our new class method.

## In Python, a class is an object

In Python, almost everything is an object. So is a class:

In [5]:
print(isinstance(Pet, object))
print(isinstance(my_degu, object))

True
True


This might look a bit confusing at first, but it really isn't. We have already used object semantics on `Pet`, and you probably did not even blink at it. The use of "`.`" to address an attribute, for instance, or the fact that we called a class method on the `Pet` class *just like* we called an instance method on a pet.

All you need to know is that you can shape a class and interact with a class in the same way as with any object. Like an object, a class has attributes holding data, and methods that you can call.

## Subclasses: the class of fishes

One thing you can do with classes is to *specialize* them by defining subclasses. For example, we can define a special kind of pet:

In [6]:
class Fish(Pet):
    def __init__(self, name, color):
        self.color = color
        super().__init__(name)
        
    def meet(self, other_fish):
        if not isinstance(other_fish, Fish):
            return
        if self.color != other_fish.color:
            if self.hungrier_than(other_fish):
                self.feed()
                other_fish.die()
            elif other_fish.hungrier_than(self):
                other_fish.feed()
                self.die()

Here's the rundown of what we did and what it means:

* `Fish` is a "subclass" of `Pet`. This means that "Fish is a Pet". Technically, it means that `Fish` "inherits" all attributes and methods from `Pet`. You'll soon see what that means exactly.
    * Nomenclature that is good to know: `Pet` is a "superclass" or "ancestor" of `Fish`. Since `Fish` is directly "derived from" `Pet`, `Pet` is also a "parent" or "base class" of `Fish`.
* `Fish.__init__()` looks a little different than `Pet.__init__()`. There is an extra parameter, and `Fish.__init__()` *calls* `Pet.__init__()` through `super()`.
* `super()` is the way to call a method in the parent class. It is a common pattern that a method that is "overridden" in a subclass (i.e. it exists in both subclass and superclass) calls `super()`. When `fish.f()` does the same as `pet.f()`, but..., then `fish.f()` probably calls `super().f()`.
* A fish can meet another fish, with predictable results.

Some code with fishes:

In [7]:
sushi = Fish(name="sushi", color="silver")
sharkie = Fish("sharkie", "blue")

print("There are {} pets.".format(len(Pet.population)))

print("first meet")
sushi.meet(sharkie)
print("time passes, and we throw in a bit of food regularly")
import random
for _ in range(11):
    Pet.advance_time()
    random.choice((sushi, sharkie)).feed()
print("second meet")
sushi.meet(sharkie)

There are 2 pets.
first meet
time passes, and we throw in a bit of food regularly
second meet
sushi dies :(


### Inheriting attributes and methods

`Fish` "inherits" all attributes and methods from `Pet`, which in turn inherits everything from `object` because we haven't specified anything else. That means that all attributes and methods of `Pet` and `object` are visible in `Fish`, unless `Fish` makes an own attribute or method of the same name. That means that for example `Fish.population` is the same as `Pet.population`. "The same as" as in "literally the same thing"!

In [8]:
# id(obj) returns the memory address of obj
print(id(Fish.population), id(Pet.population))
print(id(Fish.population) == id(Pet.population))

139818602843720 139818602843720
True


Here's how that extends to methods (beware... technicalities ahead):

In [9]:
print(id(Fish.feed), id(Pet.feed))
print(id(Fish.feed) == id(Pet.feed))

139818593643376 139818593643376
True


Wait, wasn't `feed()` an instance method?

Yes, but it is *defined* and visible in the class, simply because we made `def feed(self)` in the scope of `class Pet`. However, Python *binds* it to an object when it is called on an object. Check this:

In [10]:
nemo = Fish("nemo", "orange-ish")

nemo.hunger = 10
nemo.feed()
print(nemo.hunger)

nemo.hunger = 10
Fish.feed(nemo)
print(nemo.hunger)

0
0


`nemo.feed()` and `Fish.feed(nemo)` are the same! That is because the first form is technically just a shorthand of the second form. Whenever the Python interpreter encounters `some_object.f()`, it will effectively call `some_object.__class__.f(some_object)`.

Now you understand why every method inside a class has "`self`" as its first parameter, and why it is a good convention to name that parameter "`self`".

### Overriding attributes and methods

`Fish` defines its own `__init__()` method. We say that `Fish` "overrides" `__init__()`. Consequently, these methods are not the same in `Fish` and `Pet`:

In [11]:
print(id(Fish.__init__), id(Pet.__init__))
print(id(Fish.__init__) == id(Pet.__init__))

139818593641608 139818593642832
False


## Magic methods (\__*methodname*\__)

Some methods on classes are called "magic methods". Their name usually starts and ends with two underscores.

Magic methods are methods that have a special meaning within Python. For example, `__init__` is the method that is called when an instance of a class is constructed.

There are many other magic methods, however. You can override them in your own classes. (They are defined in `object`, so you are overriding them.)

For example, `__repr__`, which is supposed to "return a canonical string representation of an object" (whatever that is). You can get this "canonical representation" for any object by calling `repr(obj)`:

In [12]:
print(repr(nemo))
print(object.__repr__(nemo))
print(nemo)

That's not very pretty, especially when we want something like this:

In [13]:
print(Pet.population)

{<__main__.Fish object at 0x7f2a0d14d1d0>, <__main__.Fish object at 0x7f2a0d9b18d0>}


We can make this more readable by overriding `__repr__` in Pet. We do this guerilla-style here (don't do this at home!):

In [14]:
def better_pet_repr(o):
    return "{} ({})".format(o.name, o.__class__.__name__)
    
Pet.__repr__ = better_pet_repr

print(Pet.population)

{nemo (Fish), sharkie (Fish)}


# TODO
    
* some interface... but which? maybe don't use the pet example any more?
* duck typing: can be done with interface (if an object implements the interface X, it just works as an X)
* collections.abc: quickly make a subclass of one of the containers there

## Exercises:

* make a subclass Dog which satisfies the following conditions:
    * attribute 'happiness' which is initialized with 100 and decremented on advance_time.
    * method `play_fetch()` that sets happiness to 100.
    * when a dog meets another dog, both dog's happiness should be set to 100.
    * a dog is always hungrier than any other pet that is not a dog.
    * when a dog meets a fish, the dog is fed and the fish dies.
* Pets die when they are hungrier than 100.
* Fish die when they reach an age of 1000.
* Dogs die when they reach an age of 2000, or when they are less happy than 0.
* Pets are generally hungry when their hunger is > 50. With the exception of dogs, which are hungry when their hunger is > 10. Implement the method "is_hungry" wherever necessary to reflect that.
* make a classmethod `Pet.get_hungry_pets()` which returns the set of pets that are hungry.
* make a classmethod `Pet.get_population_of(klass)` which returns the set of pets that belong to the class `klass`. So `Pet.get_population_of(Fish)` should return the set of fishes.

### When classes?

* when you deal with "lots of entities of mutable data, with functions dealing with such entities"